<a href="https://colab.research.google.com/github/yahyasungur/en2tr_tr2en_machine_translation/blob/master/en2tr_sample_usage.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install -q -U tensorflow==1.14 tensor2tensor==1.13.4

In [4]:
!pip install tensorflow_datasets==3.2.1

In [5]:
import os

import tensorflow as tf
tf.logging.set_verbosity(tf.logging.ERROR)
tfe = tf.contrib.eager
tfe.enable_eager_execution() 

import numpy as np
from tensor2tensor import problems
from tensor2tensor import models
from tensor2tensor import problems
from tensor2tensor.utils import trainer_lib
from tensor2tensor.data_generators import text_encoder

from tensor2tensor.utils import t2t_model
from tensor2tensor.utils import registry

import textwrap

In [6]:
!test -d nmt-en-tr || git clone https://github.com/yahyasungur/en2tr_tr2en_machine_translation.git

In [7]:
import sys
sys.path.append("nmt-en-tr")
import nmt_en_tr

In [8]:
!wget -nc -q https://github.com/yahyasungur/en2tr_tr2en_machine_translation/releases/download/latest/en2tr.zip
!unzip -n -qq en2tr.zip

In [9]:
model_path = 'en2tr'

data_dir = os.path.join(model_path, 'data')
ckpt_dir = os.path.join(model_path, 'model')

en2tr_problem = problems.problem("translate_en_tr")
encoders = en2tr_problem.feature_encoders(data_dir)

ckpt_path = tf.train.latest_checkpoint(ckpt_dir)

In [23]:
def encode(input_str, output_str=None):
  """Input str to features dict, ready for inference"""
  inputs = encoders["inputs"].encode(input_str) + [1]  # add EOS id 
  batch_inputs = tf.reshape(inputs, [1, -1, 1])  # Make it 3D.
  return {"inputs": batch_inputs}

def decode(integers):
  """List of ints to str"""
  integers = list(np.squeeze(integers))
  if 1 in integers:
    integers = integers[:integers.index(1)]
  return encoders["inputs"].decode(np.squeeze(integers))

In [24]:
model_name = "transformer"
hparams_set = "transformer_tpu"

Modes = tf.estimator.ModeKeys

hparams = trainer_lib.create_hparams(hparams_set, data_dir=data_dir, problem_name="translate_en_tr")

translate_model = registry.model(model_name)(hparams, Modes.EVAL)

In [12]:
# Restore and translate!
def translate(inputs, beam_size = 5, alpha = 0.6, **kwargs):
  encoded_inputs = encode(inputs)
  with tfe.restore_variables_on_create(ckpt_path):
    model_output = translate_model.infer(encoded_inputs, **kwargs)["outputs"]
  if len(model_output.shape) == 2:
    return decode(model_output)
  else:
    return [decode(x) for x in model_output[0]]
  
def translate_and_display(input):
  output = translate(input)
  print('\n  '.join(textwrap.wrap("Input: {}".format(input), 80)))
  print()
  print('\n  '.join(textwrap.wrap("Output: {}".format(output), 80)))

In [13]:
inputs = "shortness of breath of difficulty breathing"

translate_and_display(inputs)

Input: shortness of breath of difficulty breathing

Output: Nefes almakta zorlanıyorum.


In [14]:
translate_and_display("be sure that the information you give is accurate and complete")

Input: be sure that the information you give is accurate and complete

Output: Verdiğin bilgilerin doğru ve eksiksiz olduğundan emin ol.


In [21]:
translate_and_display("people you have contacted during this period")

Input: people you have contacted during this period

Output: Bu süre içinde iletişim kurduğun insanlar.


In [22]:
translate_and_display("have you been in contact with anyone in the last 14 days who is experiencing these symptoms?")

Input: have you been in contact with anyone in the last 14 days who is
  experiencing these symptoms?

Output: Son 14 gün bu belirtileri yaşayan biriyle temasa geçtin mi?
